In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import shutil
import os
import re

def generate_label_mapping(root_dir, other_dir, input_subdir, output_csv):
    """
    Generate a CSV mapping input chips to corresponding segmentation maps.

    Args:
        root_dir (str or Path): Root directory containing the subdirectories for chips and segmentation maps.
        input_subdir (str): Subdirectory path for chips within the root directory.
        output_csv (str or Path): Output path for the generated CSV file.
    """
    root_dir = Path(root_dir)
    chips_orig = os.listdir(root_dir / input_subdir / "chips")

    chips = [chip.replace("chip", f"{input_subdir}/chips/chip") for chip in chips_orig]
    seg_maps = [chip.replace("chip", f"{input_subdir}/seg_maps/seg_map") for chip in chips_orig]

    df = pd.DataFrame({"Input": chips, "Label": seg_maps})
    df.to_csv(other_dir + '/' + output_csv, index=False)
    
    print(f"Number of rows is: {df.shape[0]}")
    print(f"CSV generated and saved to: {root_dir / output_csv}")
    

In [2]:
generate_label_mapping('/kaggle/input/geo-ai-hack/', '/kaggle/working/',"s2_train/s2_train/", "s2_train_ds.csv")
generate_label_mapping('/kaggle/input/geo-ai-hack/', '/kaggle/working/', "s2_test/s2_test/", "s2_test_ds.csv")

Number of rows is: 11764
CSV generated and saved to: /kaggle/input/geo-ai-hack/s2_train_ds.csv
Number of rows is: 3937
CSV generated and saved to: /kaggle/input/geo-ai-hack/s2_test_ds.csv


In [3]:
import os
import torch
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from transformers import ViTModel
import torch.optim as optim
import torch.nn as nn
import pandas as pd
from tqdm import tqdm

# Custom Dataset Class with 3 Time-Step Stacking
class CustomSegmentationDataset(Dataset):
    def __init__(self, image_paths, mask_paths=None, transform=None, is_test=False):
        self.image_paths = image_paths
        self.mask_paths = mask_paths if not is_test else None
        self.transform = transform
        self.is_test = is_test
        self.samples = []

        if not is_test:
            for i in range(len(image_paths) - 2):  # Ensure 3 consecutive images exist
                if os.path.exists(image_paths[i]) and os.path.exists(image_paths[i+1]) and os.path.exists(image_paths[i+2]) and os.path.exists(mask_paths[i]):
                    self.samples.append((image_paths[i], image_paths[i+1], image_paths[i+2], mask_paths[i]))
        else:
            for i in range(len(image_paths) - 2):
                if os.path.exists(image_paths[i]) and os.path.exists(image_paths[i+1]) and os.path.exists(image_paths[i+2]):
                    self.samples.append((image_paths[i], image_paths[i+1], image_paths[i+2], None))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img1_path, img2_path, img3_path, mask_path = self.samples[idx]
        preprocessing = Preprocessing()
        img1 = preprocessing.preprocess_image(img1_path)
        img2 = preprocessing.preprocess_image(img2_path)
        img3 = preprocessing.preprocess_image(img3_path)
        
        # Stack images as additional channels (8*3 = 24 channels total)
        image = np.concatenate([img1, img2, img3], axis=-1)
        image = torch.tensor(image.transpose(2, 0, 1), dtype=torch.float32)

        if self.is_test:
            return image, {"image_id": torch.tensor([idx])}
        else:
            mask = plt.imread(mask_path)
            if mask.ndim == 3:
                mask = mask[..., 0]
            mask = torch.tensor(mask, dtype=torch.long)
            return image, mask

# Preprocessing Class for 8-Channel Images
class Preprocessing:
    def preprocess_image(self, image_path):
        with rasterio.open(image_path) as src:
            bands = [src.read(i) for i in range(1, 7)]
        
        ndvi = self.compute_ndvi(bands[2], bands[3])
        evi = self.compute_evi(bands[3], bands[2], bands[0])
        normalized_bands = [self.normalize_band(band) for band in bands]
        image = np.stack(normalized_bands + [ndvi, evi], axis=-1)
        return image

    def normalize_band(self, band):
        return (band - np.min(band)) / (np.max(band) - np.min(band))

    def compute_ndvi(self, red, nir):
        return (nir - red) / (nir + red + 1e-6)

    def compute_evi(self, nir, red, blue, g=2.5, c1=6, c2=7.5, l=1):
        return np.clip(g * (nir - red) / (nir + c1 * red - c2 * blue + l), 0, 1)

# Vision Transformer for Segmentation
class ViTSegmentationModel(nn.Module):
    def __init__(self, num_classes=2):
        super(ViTSegmentationModel, self).__init__()
        self.vit = ViTModel.from_pretrained("facebook/dino-vits16")
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(384, 128, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, num_classes, kernel_size=2, stride=2)
        )

    def forward(self, x):
        features = self.vit(x).last_hidden_state
        features = features.permute(0, 2, 1).reshape(-1, 384, 24, 24)
        return self.decoder(features)

# Training Setup
train_csv = pd.read_csv("/kaggle/working/s2_train_ds.csv")
test_csv = pd.read_csv("/kaggle/working/s2_test_ds.csv")
train_image_paths = train_csv["Input"].tolist()
train_mask_paths = train_csv["Label"].tolist()
test_image_paths = test_csv["Input"].tolist()

train_dataset = CustomSegmentationDataset(image_paths=train_image_paths, mask_paths=train_mask_paths)
test_dataset = CustomSegmentationDataset(image_paths=test_image_paths, is_test=True)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Model Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViTSegmentationModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    model.train()
    pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
    for images, masks in pbar:
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        pbar.set_postfix({"Loss": loss.item()})
    print(f"Epoch {epoch+1} complete")

# Save Model
torch.save(model.state_dict(), "vit_segmentation.pth")

ModuleNotFoundError: No module named 'rasterio'